### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-02-27


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(2)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-02-27
Yesterday: 2025-02-25


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-02-25'
ORDER BY a.name



(22, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-02-25' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-02-21,2025-02-25,5.30,5.25,20999114,5.30,5.20,B,SET,-0.05,-0.94
1,AIMIRT,2025-02-25,2025-02-25,10.70,10.70,2094138,10.80,10.40,B,SET,0.00,0.00
2,CPNREIT,2025-02-25,2025-02-25,12.30,12.00,9280969,12.30,11.80,B,SET,-0.30,-2.44
3,DIF,2025-02-20,2025-02-25,8.10,7.80,25442910,8.20,7.80,B,SET,-0.30,-3.70
4,GVREIT,2025-02-25,2025-02-25,6.90,6.75,1697727,6.90,6.55,B,SET,-0.15,-2.17
5,JMART,2025-02-21,2025-02-25,10.40,9.70,7830782,10.50,9.70,B,SET50,-0.70,-6.73
6,KCE,2025-02-21,2025-02-25,20.50,20.50,11129282,21.20,20.50,B,SET100,0.00,0.00
7,MCS,2025-02-21,2025-02-25,7.25,7.20,2620128,7.45,7.20,B,SET,-0.05,-0.69
8,NER,2025-02-21,2025-02-25,4.96,4.88,25740120,4.96,4.88,B,SET,-0.08,-1.61
9,RCL,2025-02-21,2025-02-25,26.75,24.70,16321904,27.25,24.70,B,SET100,-2.05,-7.66


In [20]:
df.shape

(22, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,3BBIF,2025-02-21,2025-02-25,5.30,5.25,"20,999,114",5.30,5.20,-0.94%,B
1,AIMIRT,2025-02-25,2025-02-25,10.70,10.70,"2,094,138",10.80,10.40,0.00%,B
2,CPNREIT,2025-02-25,2025-02-25,12.30,12.00,"9,280,969",12.30,11.80,-2.44%,B
3,DIF,2025-02-20,2025-02-25,8.10,7.80,"25,442,910",8.20,7.80,-3.70%,B
4,GVREIT,2025-02-25,2025-02-25,6.90,6.75,"1,697,727",6.90,6.55,-2.17%,B
5,JMART,2025-02-21,2025-02-25,10.40,9.70,"7,830,782",10.50,9.70,-6.73%,B
6,KCE,2025-02-21,2025-02-25,20.50,20.50,"11,129,282",21.20,20.50,0.00%,B
7,MCS,2025-02-21,2025-02-25,7.25,7.20,"2,620,128",7.45,7.20,-0.69%,B
8,NER,2025-02-21,2025-02-25,4.96,4.88,"25,740,120",4.96,4.88,-1.61%,B
9,RCL,2025-02-21,2025-02-25,26.75,24.70,"16,321,904",27.25,24.70,-7.66%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(22, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(181, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(22, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,3BBIF,2025-02-21,2025-02-25,5.30,5.25,20999114,5.30,5.20,-0.94,B,5.30,-0.10,6758552,2025-02-27
1,AIMIRT,2025-02-25,2025-02-25,10.70,10.70,2094138,10.80,10.40,0.00,B,10.60,-0.20,80629,2025-02-27
2,CPNREIT,2025-02-25,2025-02-25,12.30,12.00,9280969,12.30,11.80,-2.44,B,11.90,-0.40,3521877,2025-02-27
3,DIF,2025-02-20,2025-02-25,8.10,7.80,25442910,8.20,7.80,-3.70,B,8.00,-0.10,11286558,2025-02-27
4,GVREIT,2025-02-25,2025-02-25,6.90,6.75,1697727,6.90,6.55,-2.17,B,6.60,0.00,118809,2025-02-27
5,JMART,2025-02-21,2025-02-25,10.40,9.70,7830782,10.50,9.70,-6.73,B,9.65,-0.45,8032901,2025-02-27
6,KCE,2025-02-21,2025-02-25,20.50,20.50,11129282,21.20,20.50,0.00,B,19.70,-0.80,8925682,2025-02-27
7,MCS,2025-02-21,2025-02-25,7.25,7.20,2620128,7.45,7.20,-0.69,B,7.15,-0.05,350385,2025-02-27
8,NER,2025-02-21,2025-02-25,4.96,4.88,25740120,4.96,4.88,-1.61,B,4.94,-0.04,6767044,2025-02-27
9,RCL,2025-02-21,2025-02-25,26.75,24.70,16321904,27.25,24.70,-7.66,B,25.00,0.00,3086722,2025-02-27


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
3BBIF    2025-02-21 2025-02-27   5.30   5.30    0.0%  20,999,114   5.30   5.20 B Update
AIMIRT   2025-02-25 2025-02-27  10.70  10.60  -0.93%   2,094,138  10.80  10.40 B Update
AWC      2025-02-20 2025-02-27   3.14   3.10  -1.27% 117,365,852   3.18   3.10 S Update
BCH      2025-02-27 2025-02-27  14.90  15.70   5.37%   9,009,982  15.80  14.90 S Insert
CPNREIT  2025-02-25 2025-02-27  12.30  11.90  -3.25%   9,280,969  12.30  11.80 B Update
DIF      2025-02-20 2025-02-27   8.10   8.00  -1.23%  25,442,910   8.20   7.80 B Update
GVREIT   2025-02-25 2025-02-27   6.90   6.60  -4.35%   1,697,727   6.90   6.55 B Update
JMART    2025-02-21 2025-02-27  10.40   9.65  -7.21%   7,830,782  10.50   9.65 B Update
JMT      2025-02-20 2025-02-27  13.90  12.10 -12.95%  25,772,894  14.20  12.10 S Update
KCE      2025-02-21 2025-02-27  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-02-27' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(22, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-02-21,2025-02-27,5.30,5.30,"41,998,228",5.30,5.20,B,SET,0.00,0.00%
1,AIMIRT,2025-02-25,2025-02-27,10.70,10.60,"4,188,276",10.80,10.40,B,SET,-0.10,-0.93%
2,CPNREIT,2025-02-25,2025-02-27,12.30,11.90,"18,561,938",12.30,11.80,B,SET,-0.40,-3.25%
3,DIF,2025-02-20,2025-02-27,8.10,8.00,"50,885,820",8.20,7.80,B,SET,-0.10,-1.23%
4,GVREIT,2025-02-25,2025-02-27,6.90,6.60,"3,395,454",6.90,6.55,B,SET,-0.30,-4.35%
5,JMART,2025-02-21,2025-02-27,10.40,9.65,"15,661,564",10.50,9.65,B,SET50,-0.75,-7.21%
6,KCE,2025-02-21,2025-02-27,20.50,19.70,"22,258,564",21.20,19.70,B,SET100,-0.80,-3.90%
7,MCS,2025-02-21,2025-02-27,7.25,7.15,"5,240,256",7.45,7.15,B,SET,-0.10,-1.38%
8,NER,2025-02-21,2025-02-27,4.96,4.94,"51,480,240",4.96,4.88,B,SET,-0.02,-0.40%
9,RCL,2025-02-21,2025-02-27,26.75,25.00,"32,643,808",27.25,24.70,B,SET100,-1.75,-6.54%


In [61]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [63]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
18,PTTGC,2025-02-20,2025-02-27,19.20,16.10,"395,728,696",20.00,16.10,S,SET50,-3.10,-16.15%
16,JMT,2025-02-20,2025-02-27,13.90,12.10,"51,545,788",14.20,12.10,S,SET50,-1.80,-12.95%
5,JMART,2025-02-21,2025-02-27,10.40,9.65,"15,661,564",10.50,9.65,B,SET50,-0.75,-7.21%
9,RCL,2025-02-21,2025-02-27,26.75,25.00,"32,643,808",27.25,24.70,B,SET100,-1.75,-6.54%
11,STA,2025-02-25,2025-02-27,16.80,15.80,"151,442,220",17.10,14.30,B,SET100,-1.00,-5.95%
13,WHAIR,2025-02-21,2025-02-27,6.00,5.70,"11,619,220",6.05,5.70,B,SET,-0.30,-5.00%


In [65]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
15,BCH,2025-02-27,2025-02-27,14.90,15.70,"9,009,982",15.80,14.90,S,SET100,0.80,5.37%


In [67]:
conlite.close()

In [69]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-27 22:07:24
